In [54]:
import os
import sys

import collections
import random
import numpy as np

import skimage
import cv2

import torch
from collections import OrderedDict
import torch.optim as optim
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval


In [55]:
model_path = 'trash_trained/final_model.pt'
model = torch.load(model_path,map_location='cpu')

In [56]:
system_dict = {};
system_dict["verbose"] = 1;
system_dict["local"] = {};
system_dict["local"]["common_size"] = 512;
system_dict["local"]["min_side"] = 608;
system_dict["local"]["max_side"] = 1024;
system_dict["local"]["mean"] = np.array([[[0.485, 0.456, 0.406]]]);
system_dict["local"]["std"] = np.array([[[0.229, 0.224, 0.225]]]);
system_dict["local"]["colors"] = [(39, 129, 113), (164, 80, 133), (83, 122, 114), (99, 81, 172), (95, 56, 104), (37, 84, 86), (14, 89, 122),
    (80, 7, 65), (10, 102, 25), (90, 185, 109), (106, 110, 132), (169, 158, 85), (188, 185, 26), (103, 1, 17),
    (82, 144, 81), (92, 7, 184), (49, 81, 155), (179, 177, 69), (93, 187, 158), (13, 39, 73), (12, 50, 60),
    (16, 179, 33), (112, 69, 165), (15, 139, 63), (33, 191, 159), (182, 173, 32), (34, 113, 133), (90, 135, 34),
    (53, 34, 86), (141, 35, 190), (6, 171, 8), (118, 76, 112), (89, 60, 55), (15, 54, 88), (112, 75, 181),
    (42, 147, 38), (138, 52, 63), (128, 65, 149), (106, 103, 24), (168, 33, 45), (28, 136, 135), (86, 91, 108),
    (52, 11, 76), (142, 6, 189), (57, 81, 168), (55, 19, 148), (182, 101, 89), (44, 65, 179), (1, 33, 26),
    (122, 164, 26), (70, 63, 134), (137, 106, 82), (120, 118, 52), (129, 74, 42), (182, 147, 112), (22, 157, 50),
    (56, 50, 20), (2, 22, 177), (156, 100, 106), (21, 35, 42), (13, 8, 121), (142, 92, 28), (45, 118, 33),
    (105, 118, 30), (7, 185, 124), (46, 34, 146), (105, 184, 169), (22, 18, 5), (147, 71, 73), (181, 64, 91),
    (31, 39, 184), (164, 179, 33), (96, 50, 18), (95, 15, 106), (113, 68, 54), (136, 116, 112), (119, 139, 130),
    (31, 139, 34), (66, 6, 127), (62, 39, 2), (49, 99, 180), (49, 119, 155), (153, 50, 183), (125, 38, 3),
    (129, 87, 143), (49, 87, 40), (128, 62, 120), (73, 85, 148), (28, 144, 118), (29, 9, 24), (175, 45, 108),
    (81, 175, 64), (178, 19, 157), (74, 188, 190), (18, 114, 2), (62, 128, 96), (21, 3, 150), (0, 6, 95),
    (2, 20, 184), (122, 37, 185)]

In [58]:
img_path = r'img1.jpg'
image_filename = os.path.basename(img_path)
img = skimage.io.imread(img_path)
image = img.astype(np.float32) / 255.;
image = (image.astype(np.float32) - system_dict["local"]["mean"]) / system_dict["local"]["std"];

rows, cols, cns = image.shape

smallest_side = min(rows, cols)

# rescale the image so the smallest side is min_side
scale = system_dict["local"]["min_side"] / smallest_side

# check if the largest side is now greater than max_side, which can happen
# when images have a large aspect ratio
largest_side = max(rows, cols)

if largest_side * scale > system_dict["local"]["max_side"]:
    scale = system_dict["local"]["max_side"]  / largest_side


# resize the image with the computed scale
image = skimage.transform.resize(image, (int(round(rows*scale)), int(round((cols*scale)))))
rows, cols, cns = image.shape

pad_w = 32 - rows%32
pad_h = 32 - cols%32

new_image = np.zeros((rows + pad_w, cols + pad_h, cns)).astype(np.float32)
new_image[:rows, :cols, :] = image.astype(np.float32)

img = torch.from_numpy(new_image)
